# Statystyczne metody przetwarzania danych 

## Laboratorium 2 - algorytm *k* Najbliższych Sąsiadów (*k*NN)


### Opis
Celem laboratorium jest implementacja klasyfikatora *k* najbliższych sąsiadów - *k*NN (*k Nearest Neighbors*) oraz zapoznanie się z innymi metrykami klasyfikacji.

### Termin
Zadanie należy wykonać tego samego dnia. 

### System oceniania

| Liczba punktów (procentowo) | Ocena |
| :----                    | ---: |
| [0-50)   | 2   |
| [50-60)  | 3   |
| [60-70)  | 3.5 |
| [70-80)  | 4   |
| [80-90)  | 4.5 |
| [90-100] | 5   |

<u>Punkty ujemne</u>

* `ocena - 0.5` jeżeli zadanie wysłano po laboratorium, ale < 7 dni; 
* `ocena - 1` jeżeli zadanie wysłano w terminie pomiędzy 7 a 14 dni;
* `ocena - 1.5` jeżeli zadanie wysłano po upływie 14 dni, ale przed ostatnim laboratorium;
* `ocena = 2` jeżeli zadanie wysłano po ostatnim laboratorium.

<u>Uwaga:</u>

Niedopuszczalne jest dzielenie się notatnikiem (plik `.ipynb`) z innymi studentami ani udostępnianie go w Internecie. Każdy student powinien pobrać notatnik samodzielnie z platformy WIKAMP.
Wysyłając zadanie potwierdasz, że wykonałeś je samodzielnie i jest to Twoja indywidualna praca a materiał przedstawiony w tej pracy jest dla Ciebie zrozumiały. Prace bardzo podobne albo grupowe będą uznawane za plagiat.


### Zbiór danych

Zbiór danych znajduje się w katalogu `dataset/*`. Jest to zmodyfikowany zbiór danych znajdujący się pod adresem: <https://archive.ics.uci.edu/ml/datasets/leaf>.

### Przesyłanie zadań

Wszystkie pliki należy spakować archiwizatorem **zip** i przesłać za pośrednictwem platformy WIKAMP. Poniżej oczekiwana zawartość archiwum:

```
+-- 📂 [IMIE.NAZWISKO].zip
    +-- 📜 Lab02.ipynb
    +-- 📂 dataset
        +-- 📜 dataset.npz
        +-- 📜 ReadMe.pdf
```

### Dodatkowe narzędzia

Dopuszczalne jest korzystanie z bibliotek: `numpy`, `pandas`, `matplotlib`.
Implementacja klasyfikatora powinna być wykonana samodzielnie (bez dodatkowych bibliotek).


### Zadanie

Należy wykonać następujące czynności w celu realizacji niniejszego zadania:
* Wczytaj dane.
* **Zaimplementuj klasyfikator *k* najbliższych sąsiadów (*k*NN)** i uruchom predykcję.
* Opisz jak zmieniają się wyniki klasyfikatora dla różnych wartości *k*, dla różnej liczby klas oraz dla różnej liczby cech.
* Wyświetl tablicę pomyłek (*confusion matrix*). W tym przypadku możesz zastosować gotową implementację z biblioteki `scikit-learn` <https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html>.
* Odszukaj przynajmniej dwie inne (niż accuracy) metryki przytatne w klasyfikacji na stronie <https://scikit-learn.org/stable/modules/model_evaluation.html> i opisz ich wyniki. Wytłumacz czym się różnią i co można z nich odczytać.
* Opisz różnicę wyników klasyfikacji obu algorytmów (NN i *k*NN)?


> UWAGA: Wszystkie wykonane elementy zadania powinny posiadać stosowne komentarze i opisy.

### PONIŻEJ WYKONAJ ZADANIE

In [1]:
# creates labels
ATTRIBUTE_NAMES = ["Class", "Specimen No.", "Eccentricity", "Aspect Ratio", "Elongation", "Solidity", "Stochastic Convexity", "Isoperimetric Factor", "Maximal Indentation Depth", "Lobedness", "Average Intensity", "Average Contrast", "Smoothness", "Third moment", "Uniformity", "Entropy"]
CLASS_NAMES = \
"""Quercus suber 
Salix atrocinera 
Populus nigra 
Alnus sp. 
Quercus robur 
Crataegus monogyna 
Ilex aquifolium 
Nerium oleander 
Betula pubescens 
Tilia tomentosa 
Acer palmatum 
Celtis sp. 
Corylus avellana 
Castanea sativa 
Populus alba 
Acer negundo 
Taxus bacatta 
Papaver sp. 
Polypolium vulgare 
Pinus sp. 
Fraxinus sp. 
Primula vulgaris 
Erodium sp. 
Bougainvillea sp. 
Arisarum vulgare 
Euonymus japonicus 
Ilex perado ssp. azorica 
Magnolia soulangeana 
Buxus sempervirens 
Urtica dioica 
Podocarpus sp. 
Acca sellowiana 
Hydrangea sp. 
Pseudosasa japonica 
Magnolia grandiflora 
Geranium sp. 
Aesculus californica 
Chelidonium majus 
Schinus terebinthifolius 
Fragaria vesca """.split("\n")
# import data
import numpy as np
with open('./dataset/dataset.npz', 'rb') as f:
    data = np.load(f, allow_pickle=True)
    train, test = data['train'], data['test']
print(train.shape) # verify shape



(2244, 16)


In [2]:
import operator
def euclidean_distance(sampleA, sampleB, attrIndices):
    sqSum = 0
    for ind in attrIndices:
        sqSum += (sampleA[ind] - sampleB[ind])**2
    sqSum = np.sqrt(sqSum)
    return sqSum

def get_most_common_class_in_distances(distances, k):
    class_hits = {}
    for sample in distances[:k]:
        current_class_hits = class_hits.setdefault(int(sample['class_index']), 0)
        class_hits[int(sample['class_index'])] = current_class_hits + 1
    return max(class_hits.items(), key=operator.itemgetter(1))[0]


def assign_k_NN(sample, trainedSamples, attrIndices, k):
    dtp = [('distance', float), ('class_index', int)]
    distances = np.array(np.zeros((trainedSamples.shape[0],1)), dtype=dtp)
    for i, trainedSample in enumerate(trainedSamples):
        distances[i]['distance'] = euclidean_distance(sample, trainedSamples[i], attrIndices)
        distances[i]['class_index'] = trainedSamples[i][0]
    distances.sort(0, order='distance') # sort ascending
    return get_most_common_class_in_distances(distances, k)

#assign_k_NN(test[0], train, [4,5], 3)
CLASS_COUNT = 40

classIndexes = []
for i in range(CLASS_COUNT):
    classIndexes.append([])
for i, sample in enumerate(train):
    classIndexes[int(sample[0])-1].append(int(i))
"""
for i, sampleClass in enumerate(classIndexes):
    print(i,len(sampleClass))
"""
classIndexesTest = []
for i in range(CLASS_COUNT):
    classIndexesTest.append([])
for i, sample in enumerate(test):
    classIndexesTest[int(sample[0])-1].append(int(i))
    

from sklearn.metrics import confusion_matrix

def run_knn_test(classes, attributes, k):
    chosen_train_samples = np.concatenate(
        [train[classIndexes[i],:] for i in classes],
        axis=0
    )
    chosen_test_samples = np.concatenate(
        [test[classIndexesTest[i],:] for i in classes],
        axis=0
    )
    #print(f"Train samples: {chosen_train_samples.shape}. Test samples: {chosen_test_samples.shape}")
    test_classes = [sample[0] for sample in chosen_test_samples]
    test_nearest_neigbours = \
        [int(assign_k_NN(test_sample, chosen_train_samples, attributes, k)) for test_sample in chosen_test_samples]

    totalTestSamples = chosen_test_samples.shape[0]
    correctSamples = 0
    for i, sample in enumerate(chosen_test_samples):
        if(test_nearest_neigbours[i] == sample[0]): correctSamples+=1

    return {
        "total_samples": totalTestSamples,
        "correct_samples": correctSamples,
        "confusion_matrix": confusion_matrix(test_classes, test_nearest_neigbours)
    }
    

In [11]:
def print_test_result(k, r:dict, matrix_class_indexes=None):
    """ Prints formatted result dict of run_knn_test()
    """
    print("{}NN: {} correct out of {} total. Accuracy: {:.2f}%".format(k, r['correct_samples'], r['total_samples'], r['correct_samples']/r['total_samples']*100))
    if(matrix_class_indexes):
        print("{:8.8s}".format(""),end='|')
        for index in matrix_class_indexes: #print top row
            print("{:.8s}".format(CLASS_NAMES[index]),end='|')
        print()
        for i, row in enumerate(r["confusion_matrix"]): #print rows
            print("{:.8s}".format(CLASS_NAMES[i]),end='|')
            for col in row:
                print("{:8d}".format(int(col)),end='|')
            print()
                
    

In [12]:
        
        
print("\nTesting if 1-NN is lab01 NN (Zadanie 1.9):")
PARAM_K = 1
PARAM_CLASSES = [2, 4]
PARAM_ATTRIBUTES = [4,15]
print(run_knn_test(PARAM_CLASSES, PARAM_ATTRIBUTES, PARAM_K))
print("Expected accuracy: 100%")

print("\nTesting if 1-NN is lab01 NN (Zadanie 1.9):")
PARAM_K = 1
PARAM_CLASSES = [22,23,24,25,26]
PARAM_ATTRIBUTES = [4,7,9,11,13]
print(run_knn_test(PARAM_CLASSES, PARAM_ATTRIBUTES, PARAM_K))
print("Expected accuracy: 59.09%\n")


print("\nTesting 1/5/10/25NN with 8 classes and 5 attributes")
PARAM_CLASSES = [0,1,2,3,4,5,6,7]
PARAM_ATTRIBUTES = [4,7,9,11,13] 
print_test_result(1, run_knn_test(PARAM_CLASSES, PARAM_ATTRIBUTES, 1))
PARAM_K = 5
print_test_result(PARAM_K, run_knn_test(PARAM_CLASSES, PARAM_ATTRIBUTES, PARAM_K))
PARAM_K = 10
print_test_result(PARAM_K, run_knn_test(PARAM_CLASSES, PARAM_ATTRIBUTES, PARAM_K))


print("\nTesting 1/5/10/25NN with 12 classes and 5 attributes")
PARAM_CLASSES = [0,1,2,3,4,5,6,7, 8,9,10,11]
PARAM_ATTRIBUTES = [4,7,9,11,13] 
print_test_result(1, run_knn_test(PARAM_CLASSES, PARAM_ATTRIBUTES, 1))
PARAM_K = 5
print_test_result(PARAM_K, run_knn_test(PARAM_CLASSES, PARAM_ATTRIBUTES, PARAM_K))
PARAM_K = 10
print_test_result(PARAM_K, run_knn_test(PARAM_CLASSES, PARAM_ATTRIBUTES, PARAM_K))


print("\nTesting 1/5/10/25NN with 4 classes and 5 attributes")
PARAM_CLASSES = [0,1,2,3]
PARAM_ATTRIBUTES = [4,7,9,11,13] 
print_test_result(1, run_knn_test(PARAM_CLASSES, PARAM_ATTRIBUTES, 1))
PARAM_K = 5
print_test_result(PARAM_K, run_knn_test(PARAM_CLASSES, PARAM_ATTRIBUTES, PARAM_K))
PARAM_K = 10
print_test_result(PARAM_K, run_knn_test(PARAM_CLASSES, PARAM_ATTRIBUTES, PARAM_K))


print("\nTesting 1/5/10/25NN with 8 classes and 10 attributes")
PARAM_CLASSES = [0,1,2,3]
PARAM_ATTRIBUTES = [4,5,6,7,8,9,10,11,12,13] 
print_test_result(1, run_knn_test(PARAM_CLASSES, PARAM_ATTRIBUTES, 1))
PARAM_K = 5
print_test_result(PARAM_K, run_knn_test(PARAM_CLASSES, PARAM_ATTRIBUTES, PARAM_K))
PARAM_K = 10
print_test_result(PARAM_K, run_knn_test(PARAM_CLASSES, PARAM_ATTRIBUTES, PARAM_K))



Testing if 1-NN is lab01 NN (Zadanie 1.9):
{'total_samples': 110, 'correct_samples': 110, 'confusion_matrix': array([[55,  0],
       [ 0, 55]], dtype=int64)}
Expected accuracy: 100%

Testing if 1-NN is lab01 NN (Zadanie 1.9):
{'total_samples': 242, 'correct_samples': 143, 'confusion_matrix': array([[41,  2,  0,  1,  0],
       [17, 20,  0, 17, 12],
       [ 0,  0, 33,  0,  0],
       [ 2,  5,  0, 20,  6],
       [ 0, 17,  0, 20, 29]], dtype=int64)}
Expected accuracy: 59.09%


Testing 1/5/10/25NN with 8 classes and 5 attributes
1NN: 344 correct out of 429 total. Accuracy: 80.19%
5NN: 347 correct out of 429 total. Accuracy: 80.89%
10NN: 355 correct out of 429 total. Accuracy: 82.75%

Testing 1/5/10/25NN with 12 classes and 5 attributes
1NN: 455 correct out of 660 total. Accuracy: 68.94%
5NN: 458 correct out of 660 total. Accuracy: 69.39%
10NN: 453 correct out of 660 total. Accuracy: 68.64%

Testing 1/5/10/25NN with 4 classes and 5 attributes
1NN: 125 correct out of 176 total. Accuracy:

Zastosowanie metody k-NN (k>1) generalnie doprowadziło do zauważalnej poprawy wyników klasyfikacji. Metoda ta doprowadziła przede wszystkim do wyeliminowania tych przypadków w których metoda 1-NN zaklasyfikowałaby badaną próbkę do błędnej klasy na podstawie próbki nienaturalnie oddalonej od reszty próbek z klasy. 

Przy jednej z konfiguracji (12 klas, 5 atrybutów) użycie metody 10-NN dało gorsze rezultaty niż metod 5-NN, i 1-NN. Parametr k musi zostać dobrany odpowiednio do badanego zestawu i jego zwiększanie nie zawsze doprowadzi do poprawy celności klasyfikacji, zwłaszcza jeśli wybrane do klasyfikacji cechy nie są dostatecznie dyskryminujące.

Taka sytuacja prawdopodobnie wydarzyła się w ostatniej konfiguracji (8 klas, 10 atrybutów), gdzie mimo większej ilości branych pod uwagę cech wyniki klasyfikacji są gorsze niż w konfiguracji korzystającej z mniejszej ilości atrybutów (5), przy tej samej ilości klas.

In [22]:
def print_test_result(k, r:dict, matrix_class_indexes=None):
    print("{}NN: {} correct out of {} total. Accuracy: {:.2f}%".format(k, r['correct_samples'], r['total_samples'], r['correct_samples']/r['total_samples']*100))
    if(matrix_class_indexes):
        print("{:8.8s}".format(""),end='|')
        for index in matrix_class_indexes: #print top row
            print("{:.8s}".format(CLASS_NAMES[index]),end='|')
        print()
        for i, row in enumerate(r["confusion_matrix"]): #print rows
            print("{:.8s}".format(CLASS_NAMES[i]),end='|')
            for col in row:
                print("{:8d}".format(int(col)),end='|')
            print()
                
PARAM_K = 5
PARAM_CLASSES = [0,1,2,3,4,5,6,7]
PARAM_ATTRIBUTES = [4,7,9,11,13] 
print_test_result(PARAM_K, run_knn_test(PARAM_CLASSES, PARAM_ATTRIBUTES, PARAM_K), PARAM_CLASSES)

5NN: 347 correct out of 429 total. Accuracy: 80.89%
        |Quercus |Salix at|Populus |Alnus sp|Quercus |Crataegu|Ilex aqu|Nerium o|
Quercus |      47|       1|       0|       7|       0|       0|       0|       0|
Salix at|      11|      33|       0|       0|       0|       0|       0|       0|
Populus |       9|       2|      33|      11|       0|       0|       0|       0|
Alnus sp|       5|       0|       0|      17|       0|       0|       0|       0|
Quercus |       0|       0|       0|       0|      53|       0|       2|       0|
Crataegu|       0|       0|       0|       0|       0|      44|       0|       0|
Ilex aqu|       3|      17|       0|       6|       4|       0|      36|       0|
Nerium o|       0|       0|       0|       0|       4|       0|       0|      84|


W analizowanym przypadku Salix atrocinerea był często błędnie klasyfikowany jako Quercus robur; a Ilex aquifolium jako Salix atrocinerea. Liście posiadają m.in dość podobny kształt pod względem długości i szerokości co jest prawdopodobnie przyczyną tego stanu rzeczy. 
<img src="https://www.sorianatural.com/storage/img/F0000001472_plantas_web_ok_oak.jpg" style="height:200px;"> 
Quercus robur
<img src="https://www.arbolapp.es/imagenes/especies/especie_079_01.jpg" style="height:200px;"> 
Salix atrocinerea
<img src="https://www.monaconatureencyclopedia.com/wp-content/uploads/2008/08/le_foglie_ed_i_frutti_rossi_persistenti_d_inverno_fanno_allegria_e_l_ilex_aquifolium_e_sempre_stato_considerato_un_porta_fortuna.jpg" style="height:200px;"> 
Ilex aquifolium

<h3>F1</h3>
Recall jest to procent poprawnych klasyfikacji obiektów danej klasy w stosunku do liczby obiektów z tej klasy. Daje informację jak skutecznie klasyfikator wykrywa obiekty danej klasy.

Precision jest to procent poprawnych klasyfikacji obiektów danej klasy w stosunku do całej liczby obiektów które zostały zaklasyfikowane do danej klasy. Daje informację ile z obiektów uznanych za daną klasę faktycznie do niej należy.

F1 jest metryką mającą uwzględnić obie te wartości w jednej liczbie. 
Do jej obliczenia wykorzystana jest średnia harmoniczna:
$$F_{\beta}=(1+\beta^2)\bullet \frac{precision \bullet recall}{(\beta^2 \bullet precision)+recall}$$
Dla $$\beta==1$$ oba parametry są tak samo ważne i ze względu na wykorzystanie średniej harmonicznej jeśli jeden z nich będzie bliski zera cała średnia również będzię osiągać niskie wartości. Zwiększenie $$\beta$$ stawia większą wagę na recall a zmniejszenie poniżej jedności na precision.

Wykorzystanie F1 pozwala wykryć sytuacje w której klasa mniej liczniej reprezentowana w zbiorze testowym nie osiąga takiego samego poziomu celności klasyfikacji jak inne.

<h3>Balanced Accuracy</h3>
Specificity jest ilorazem liczby próbek niezaklasyfikowanych do analizowanej klasy do całej liczby próbek nienależących do tej klasy. Jest miarą tego jak skutecznie odrzucane są obiekty pochodzące z innych klas.

Balanced Accuracy jest średnią arytmetyczną wartości recall i specificity.
$$balanced accuracy=\frac{recall+specificity}{2}$$
Spełnia to podobne zadanie co F1. W przypadku niezbalansowanych klas pozwala na wykrycie problemów z dyskryminacją tych mniej licznych. Różnica wynika z tego że F1 nie bierze pod uwagę próbek niezaklasyfikowanych do danej klasy, balanced accuracy sprawdzi się lepiej jeśli zależy nam na uwzględnieniu tego jak skutecznie klasyfikator odrzuca prawdziwie błędne klasy.


In [ ]:

if(True): # get_most_common_class_in_distances
    print("BEGIN TEST")
    dtp = [('distance', float), ('class_index', int)]
    distances = np.array([(10,1),(20,2),(21,2),(32,3),(30,3),(50,5),(31,3)], dtype=dtp)
    distances[:].sort(0, order='distance') # sort descending
    print(distances)
    print(get_most_common_class_in_distances(distances, 1))
    print(get_most_common_class_in_distances(distances, 2))
    print(get_most_common_class_in_distances(distances, 3))
    print(get_most_common_class_in_distances(distances, 6))
    print("END TEST")